In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

In [2]:
df_citylab_rdy = pd.read_csv("Citylab-Berlin_Projekte.csv", sep=";")
df_citylab_rdy.head()

,"Projektname,Einsatzbereich,Web-Link,Ansprechperson,Email,Status,Kurzzusammenfassung,Webseite-Link,Quelle,Organisation,Art"
0,"BärGPT,""['Digitale Zusammenarbeit', 'Neue Tech..."
1,"Data Hub Berlin,""['Innovative Verwaltung', 'Of..."
2,"Partizipation Digital,""['Digitale Zusammenarbe..."
3,"GovTech TestLAB,""['Digitale Zusammenarbeit', '..."
4,"Stadtlabor2Go,""['Digitale Zusammenarbeit', 'In..."


***
## Main Page

In [3]:
url = 'https://citylab-berlin.org/de/projects/'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

In [4]:
first_chunks = soup.find_all('div', class_='wpgb-masonry')

laufend_chunk = first_chunks[0].find_all('article')
abgeschlossen_chunk = first_chunks[1].find_all('article')

In [5]:
laufend_dict = {}

for tile in laufend_chunk: 
    sub_domain_link = tile.find('a', class_="wpgb-card-layer-link").get('href')
    h3 = tile.find('h3', class_="wpgb-block-3 wpgb-idle-scheme-1")
    if h3:
        textlink = h3.find('a')
        if textlink:
            title = textlink.text
    
    categeory_texts = []
    cat = tile.find('div', class_="wpgb-block-2")
    if cat: 
        categories = cat.find_all('a')
        if categories:
            for category in categories:
                categeory_text = category.text
                categeory_texts.append(categeory_text)
    
    laufend_dict[title] = {
        "sub-link" : sub_domain_link,
        "Einsatzbereich" : categeory_texts
    } 

for key in list(laufend_dict.keys()):
    split_string = re.split(r'[\\]', key)
    if split_string:  # Check if the split string has elements
        new_key = split_string[0].strip()
    laufend_dict[new_key] = laufend_dict.pop(key)

laufend_dict

{'BärGPT': {'sub-link': 'https://citylab-berlin.org/de/projects/baergpt/',
  'Einsatzbereich': ['Digitale Zusammenarbeit', 'Neue Technologien']},
 'Data Hub Berlin': {'sub-link': 'https://citylab-berlin.org/de/projects/data-hub-berlin/',
  'Einsatzbereich': ['Innovative Verwaltung', 'Offene Daten', 'Open Source']},
 'Partizipation Digital': {'sub-link': 'https://citylab-berlin.org/de/projects/partizipation-digital/',
  'Einsatzbereich': ['Digitale Zusammenarbeit', 'Innovative Verwaltung']},
 'GovTech TestLAB': {'sub-link': 'https://citylab-berlin.org/de/projects/govtech-testlab/',
  'Einsatzbereich': ['Digitale Zusammenarbeit',
   'Innovative Verwaltung',
   'Open Source']},
 'Stadtlabor2Go': {'sub-link': 'https://citylab-berlin.org/de/projects/stadtlabor2go/',
  'Einsatzbereich': ['Digitale Zusammenarbeit',
   'Innovative Verwaltung',
   'Offene Daten, offene Städte']},
 'Fairgnügen': {'sub-link': 'https://citylab-berlin.org/de/projects/fairgnuegen/',
  'Einsatzbereich': ['Digitale Zu

In [6]:
abgeschlossen_dict = {}

for tile in abgeschlossen_chunk: 
    sub_domain_link = tile.find('a', class_="wpgb-card-layer-link").get('href')
    h3 = tile.find('h3', class_="wpgb-block-3 wpgb-idle-scheme-1")
    if h3:
        textlink = h3.find('a')
        if textlink:
            title = textlink.text

    categeory_texts = []
    cat = tile.find('div', class_="wpgb-block-2")
    if cat: 
        categories = cat.find_all('a')
        if categories:
            for category in categories:
                categeory_text = category.text
                categeory_texts.append(categeory_text)

    abgeschlossen_dict[title] = {
        "sub-link" : sub_domain_link,
        "Einsatzbereich" : categeory_texts
    } 

for key in list(abgeschlossen_dict.keys()):
    split_string = re.split(r'[\–\-\:]', key)
    if split_string:  # Check if the split string has elements
        new_key = split_string[0].strip()
    abgeschlossen_dict[new_key] = abgeschlossen_dict.pop(key)

abgeschlossen_dict

{'Digital Vereint': {'sub-link': 'https://citylab-berlin.org/de/projects/digitalvereint/',
  'Einsatzbereich': ['Smart Cities für alle']},
 'Quantified Trees (Qtrees)': {'sub-link': 'https://citylab-berlin.org/de/projects/qtrees/',
  'Einsatzbereich': ['Energie und Nachhaltigkeit']},
 'Verwaltungsdashboard': {'sub-link': 'https://citylab-berlin.org/de/projects/afs-dashboard/',
  'Einsatzbereich': ['Innovative Verwaltung']},
 'Stadtpuls': {'sub-link': 'https://citylab-berlin.org/de/projects/stadtpuls/',
  'Einsatzbereich': ['Offene Daten, offene Städte', 'Smart Cities für alle']},
 'Pixelwelt': {'sub-link': 'https://citylab-berlin.org/de/projects/pixelwelt/',
  'Einsatzbereich': ['Digitale Zusammenarbeit']},
 'Prototypenwerkstatt': {'sub-link': 'https://citylab-berlin.org/de/projects/prototypenwerkstatt/',
  'Einsatzbereich': ['Innovative Verwaltung']},
 'Prozessanalyse Radinfrastruktur (PARI)': {'sub-link': 'https://citylab-berlin.org/de/projects/prozessanalyse-radinfrastruktur-pari/',

*** 
## Subdomains

In [7]:
import json

abgeschlossen_soup_dict = {}

for key, values in abgeschlossen_dict.items():
    subdomain_url = values['sub-link']
    response_subdomain = requests.get(subdomain_url)
    subsoup = BeautifulSoup(response_subdomain.text, 'html.parser')
    abgeschlossen_soup_dict[key] = str(subsoup)

In [8]:
with open('abgeschlossen_soup_dict.json', 'w', encoding='utf-8') as f:
    json.dump(abgeschlossen_soup_dict, f)

In [9]:
laufend_soup_dict = {}

for key, values in laufend_dict.items():
    subdomain_url = values['sub-link']
    response_subdomain = requests.get(subdomain_url)
    subsoup = BeautifulSoup(response_subdomain.text, 'html.parser')
    laufend_soup_dict[key] = str(subsoup)

In [10]:
with open('laufend_soup_dict.json', 'w', encoding='utf-8') as f:
    json.dump(laufend_soup_dict, f)

In [11]:
for key, values in abgeschlossen_dict.items():
    subsoup = BeautifulSoup(abgeschlossen_soup_dict[key], 'html.parser')

    footer_div = subsoup.find('div', class_="site-footer")
    if footer_div:
        footer_div.extract()
    
    # Initialize variables to avoid NameError in the matrix
    web_link = None
    kontakt = None
    email = None
    status = None
    text = None
    
    # Extract web link - primary method
    link_header = subsoup.find('h2', class_="wp-block-heading", id="webseite")
    if link_header:
        web_link_container = link_header.find_next('a')
        if web_link_container:
            web_link = web_link_container.get('href')
    
    # Fallback to sub-link if primary method fails
    if web_link is None:
        web_link = abgeschlossen_dict[key]['sub-link']
    
    kontakt_header = subsoup.find(['h1', 'h2', 'h3', 'h4', 'h5', 'h6'], string=lambda text: text and 'kontakt' in text.strip().lower())
    kontakt_container = kontakt_header.find_next('p')

    contacts = kontakt_container.find_all('a')
    emails = []
    kontakte = []
    for contact in contacts:
        contact_text = contact.get_text(strip=True)
        kontakte.append(contact_text)

        href = contact.get('href')
        # Special case for "CityLAB Berlin" - hardcoded like a corpo backdoor
        if contact_text == "CityLAB Berlin":
            emails.append("info@citylab-berlin.org")
        # Extract email from mailto: links
        elif href.startswith('mailto:'):
            email = href.replace('mailto:', '').strip()
            emails.append(email)
        # Capture any other hrefs too, bredrin
        elif href:
            emails.append(href)

    status = "Projekt Abgeschlossen"

    content = subsoup.find('div', class_="entry-content")
    if content: 
        warum_text = content.find_next('p')
        if warum_text:
            text = warum_text.get_text(strip=True)
    
    # Update the data structure
    values.update({
        "Webseite-Link": web_link,
        "Ansprechperson": kontakte,
        "Email": emails,
        "Status": status,
        "Kurzzusammenfassung": text
    })

In [ ]:
for key, values in laufend_dict.items():
    subsoup = BeautifulSoup(laufend_soup_dict[key], 'html.parser')

    footer_div = subsoup.find('div', class_="site-footer")
    if footer_div:
        footer_div.extract()
    
    # Initialize variables to avoid NameError in the matrix
    web_link = None
    kontakt = None
    email = None
    status = None
    text = None
    
    # Extract web link - primary method
    link_header = subsoup.find('h2', class_="wp-block-heading", id="webseite")
    if link_header:
        web_link_container = link_header.find_next('a')
        if web_link_container:
            web_link = web_link_container.get('href')
    
    # Fallback to sub-link if primary method fails
    if web_link is None:
        web_link = laufend_dict[key]['sub-link']
    
    kontakt_header = subsoup.find(['h1', 'h2', 'h3', 'h4', 'h5', 'h6'], string=lambda text: text and 'kontakt' in text.strip().lower())
    if kontakt_header:
        kontakt_container = kontakt_header.find_next('p')

    contacts = kontakt_container.find_all('a')
    emails = []
    kontakte = []
    for contact in contacts:
        contact_text = contact.get_text(strip=True)
        kontakte.append(contact_text)

        href = contact.get('href')
        # Special case for "CityLAB Berlin" - hardcoded like a corpo backdoor
        if contact_text == "CityLAB Berlin":
            emails.append("info@citylab-berlin.org")
        # Extract email from mailto: links
        elif href.startswith('mailto:'):
            email = href.replace('mailto:', '').strip()
            emails.append(email)
        # Capture any other hrefs too, bredrin
        elif href:
            emails.append(href)

    status = "Laufend"

    content = subsoup.find('div', class_="entry-content")
    if content: 
        warum_text = content.find_next('p')
        if warum_text:
            text = warum_text.get_text(strip=True)
    
    # Update the data structure
    values.update({
        "Webseite-Link": web_link,
        "Ansprechperson": kontakte,
        "Email": emails,
        "Status": status,
        "Kurzzusammenfassung": text
    })

In [13]:
with open('laufend_dict_working.json', 'w', encoding='utf-8') as f:
    json.dump(laufend_dict, f, ensure_ascii=False, indent=4)

with open('abgeschlossen_dict_working.json', 'w', encoding='utf-8') as f:
    json.dump(abgeschlossen_dict, f, ensure_ascii=False, indent=4)

In [26]:
df_abgeschlossen = pd.DataFrame()

df_abgeschlossen["Projektname"] = abgeschlossen_dict.keys()
values_df = pd.DataFrame(list(abgeschlossen_dict.values()))
df_abgeschlossen = pd.concat([df_abgeschlossen, values_df], axis=1)

In [ ]:
df_abgeschlossen

,Projektname,sub-link,Einsatzbereich,Webseite-Link,Ansprechperson,Email,Status,Kurzzusammenfassung
0,Digital Vereint,https://citylab-berlin.org/de/projects/digital...,[Smart Cities für alle],https://citylab-berlin.org/de/projects/digital...,[[email protected]],[/cdn-cgi/l/email-protection#a4cfcbcad0c5cfd0e...,Projekt Abgeschlossen,Für die vielen engagierten Vereine und Initiat...
1,Quantified Trees (Qtrees),https://citylab-berlin.org/de/projects/qtrees/,[Energie und Nachhaltigkeit],https://citylab-berlin.org/de/projects/qtrees/,"[Julia Zimmermann, Myrian Rigal]",[/cdn-cgi/l/email-protection#a5dfccc8c8c0d7c8c...,Projekt Abgeschlossen,"Gefördert vom Bundesministerium für Umwelt, Na..."
2,Verwaltungsdashboard,https://citylab-berlin.org/de/projects/afs-das...,[Innovative Verwaltung],https://citylab-berlin.org/de/projects/afs-das...,[Tobias Witt],[/cdn-cgi/l/email-protection#e89f819c9ca89c8d8...,Projekt Abgeschlossen,In vielen Bereichen der Berliner Verwaltung we...
3,Stadtpuls,https://citylab-berlin.org/de/projects/stadtpuls/,"[Offene Daten, offene Städte, Smart Cities für...",http://www.stadtpuls.com,[],[],Projekt Abgeschlossen,"Stadtpuls ist eine offene IoT-Datenplattform, ..."
4,Pixelwelt,https://citylab-berlin.org/de/projects/pixelwelt/,[Digitale Zusammenarbeit],https://citylab-berlin.org/de/projects/pixelwelt/,[[email protected]],[/cdn-cgi/l/email-protection#1d74737b725d7e746...,Projekt Abgeschlossen,"Super Mario war gestern, jetzt gibts das Super..."
5,Prototypenwerkstatt,https://citylab-berlin.org/de/projects/prototy...,[Innovative Verwaltung],https://citylab-berlin.org/de/projects/prototy...,[Ingo Hinterding],[/cdn-cgi/l/email-protection#492120273d2c3b2d2...,Projekt Abgeschlossen,Das Projekt „Prototypenwerkstatt“ ist eine Koo...
6,Prozessanalyse Radinfrastruktur (PARI),https://citylab-berlin.org/de/projects/prozess...,"[Energie und Nachhaltigkeit, Offene Daten, off...",https://citylab-berlin.org/de/projects/prozess...,[Peter Broytman],[/cdn-cgi/l/email-protection#dfafbaabbaadf1bda...,Projekt Abgeschlossen,"Radfahren ist gut für das Klima, gegen Stau un..."
7,FixMyBerlin,https://citylab-berlin.org/de/projects/digital...,[Innovative Verwaltung],https://citylab-berlin.org/de/projects/digital...,"[[email protected], fixmyberlin.de]",[/cdn-cgi/l/email-protection#741c1118181b34121...,Projekt Abgeschlossen,FixMyBerlin begleitet die Berliner Verwaltung ...
8,Berliner Erfrischungskarte,https://citylab-berlin.org/de/projects/schatten/,"[Offene Daten, offene Städte]",https://citylab-berlin.org/de/projects/schatten/,[Lisa Stubert],[/cdn-cgi/l/email-protection#1975706a78376a6d6...,Projekt Abgeschlossen,Klimatische Unterschiede in der Stadt besser v...
9,Clair Berlin,https://citylab-berlin.org/de/projects/clair/,[Energie und Nachhaltigkeit],https://citylab-berlin.org/de/projects/clair/,"[, Clair Berlin, CityLAB Berlin]",[/cdn-cgi/l/email-protection#3d4e58545f58517d4...,Projekt Abgeschlossen,Clair Berlin war eine gemeinwohlorientierte In...


In [28]:
df_laufend = pd.DataFrame()

df_laufend["Projektname"] = laufend_dict.keys()
values_df = pd.DataFrame(list(laufend_dict.values()))
df_laufend = pd.concat([df_laufend, values_df], axis=1)

In [30]:
df_citylab_working = pd.DataFrame()

df_citylab_working = pd.concat([df_laufend, df_abgeschlossen])
df_citylab_working.reset_index(drop=True, inplace=True)

In [31]:
df_citylab_working

,Projektname,sub-link,Einsatzbereich,Webseite-Link,Ansprechperson,Email,Status,Kurzzusammenfassung
0,BärGPT,https://citylab-berlin.org/de/projects/baergpt/,"[Digitale Zusammenarbeit, Neue Technologien]",https://citylab-berlin.org/de/projects/baergpt/,[[email protected]],[/cdn-cgi/l/email-protection],Laufend,Bis 2030 gehen voraussichtlich rund 30 Prozent...
1,Data Hub Berlin,https://citylab-berlin.org/de/projects/data-hu...,"[Innovative Verwaltung, Offene Daten, Open Sou...",https://citylab-berlin.org/de/projects/data-hu...,[[email protected]],[/cdn-cgi/l/email-protection#15797c66743b66616...,Laufend,"Luftqualität, Stauwarnungen oder digitale Serv..."
2,Partizipation Digital,https://citylab-berlin.org/de/projects/partizi...,"[Digitale Zusammenarbeit, Innovative Verwaltung]",https://citylab-berlin.org/de/projects/partizi...,[[email protected]],[/cdn-cgi/l/email-protection#15797c66743b66616...,Laufend,Im Rahmen unseres Fokusthemas „Ankommen in Ber...
3,GovTech TestLAB,https://citylab-berlin.org/de/projects/govtech...,"[Digitale Zusammenarbeit, Innovative Verwaltun...",https://citylab-berlin.org/de/projects/govtech...,[Markus Sperl (Projektmanager Smart City & Ver...,[https://www.linkedin.com/in/markus-johannes-s...,Laufend,Die Herausforderungen der Berliner Verwaltungs...
4,Stadtlabor2Go,https://citylab-berlin.org/de/projects/stadtla...,"[Digitale Zusammenarbeit, Innovative Verwaltun...",https://citylab-berlin.org/de/projects/stadtla...,[],[],Laufend,"Viele Kommunen stehen vor der Herausforderung,..."
5,Fairgnügen,https://citylab-berlin.org/de/projects/fairgnu...,"[Digitale Zusammenarbeit, Innovative Verwaltun...",https://citylab-berlin.org/de/projects/fairgnu...,[],[],Laufend,Berlin bietet eine Fülle an ermäßigten oder ko...
6,Kiezlabor,https://citylab-berlin.org/de/projects/kiezlabor/,"[Digitale Zusammenarbeit, Innovative Verwaltun...",https://citylab-berlin.org/de/projects/kiezlabor/,[],[],Laufend,None
7,Gieß den Kiez,https://citylab-berlin.org/de/projects/giess-d...,"[Energie und Nachhaltigkeit, Offene Daten, off...",https://citylab-berlin.org/de/projects/giess-d...,"[www.giessdenkiez.de, [email protected], Slack...","[https://www.giessdenkiez.de/, /cdn-cgi/l/emai...",Laufend,Die Folgen der Klimakrise in den Städten sind ...
8,Parla,https://citylab-berlin.org/de/projects/parla/,"[Digitale Zusammenarbeit, Innovative Verwaltun...",https://citylab-berlin.org/de/projects/parla/,[Ingo Hinterding],[/cdn-cgi/l/email-protection#fc95929b93d294959...,Laufend,Welche Nachhaltigkeitsziele verfolgt der Berli...
9,KlimaDashboard Xhain,https://citylab-berlin.org/de/projects/klimada...,"[Digitale Zusammenarbeit, Energie und Nachhalt...",https://citylab-berlin.org/de/projects/klimada...,[[email protected]],[/cdn-cgi/l/email-protection],Laufend,Der Berliner Senat verfolgt eine ambitionierte...


In [32]:
df_citylab_working["Organisation"] = "CityLAB-Berlin"
df_citylab_working["Art"] = None
df_citylab_working.rename(columns={'sub-link': 'Quelle'}, inplace=True)

In [33]:
df_citylab_working.to_csv('Citylab-Berlin_Projekte.csv', index=False)